# **Laboratorijska vježba 4**: Zadaci za samostalni rad

## **Podaci o pacijentima s kardiovaskularnim bolestima**

U ovoj laboratorijskoj vježbi istražit ćemo skup podataka koji sadrži informacije o pacijentima s kardiovaskularnim bolestima. Skup podataka sadrži sljedeće značajke:

* DEATH_EVENT - je li pacijent preminuo tijekom razdoblja praćenja
* sex - muško(1) ili žensko (0)
* anaemia - smanjenje crvenih krvnih stanica
* smoking
* diabetes - boluje li pacijent od dijabetesa
* high_blood_pressure - boluje li pacijent od hipertenzije
* age - dob pacijenta
* creatinine_phosphokinase - razina CPK enzima u krvi
* ejection_fraction - postotak krvi koji napušta srce u svakoj kontrakciji
* platelets - razina trombocita u krvi 
* serum_creatinine - razina kreatinina u krvi
* serum_sodium - razina serumskog natrija u krvi
* time - trajanje razdoblja praćenja (broj dana u bolnici)


In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [5]:
data_folder = './Data/'
df = pd.read_csv(data_folder + 'heart_failure_clinical_records_dataset.csv')
df.sample(10)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
72,85.000,0,5882,0,35,0,243000.00,1.0,132,1,1,72,1
288,65.000,0,892,1,35,0,263358.03,1.1,142,0,0,256,0
267,56.000,1,135,1,38,0,133000.00,1.7,140,1,0,244,0
99,63.000,1,61,1,40,0,221000.00,1.1,140,0,0,86,0
58,49.000,0,789,0,20,1,319000.00,1.1,136,1,1,55,1
83,79.000,1,55,0,50,1,172000.00,1.8,133,1,0,78,0
252,50.000,0,245,0,45,1,274000.00,1.0,133,1,0,215,0
185,60.667,1,104,1,30,0,389000.00,1.5,136,1,0,171,1
93,60.000,1,154,0,25,0,210000.00,1.7,135,1,0,82,1
137,68.000,1,646,0,25,0,305000.00,2.1,130,1,0,108,0


In [3]:
df.corr()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
age,1.000000,0.088006,-0.081584,-0.101012,0.060098,0.093289,-0.052354,0.159187,-0.045966,0.065430,0.018668,-0.224068,0.253729
anaemia,0.088006,1.000000,-0.190741,-0.012729,0.031557,0.038182,-0.043786,0.052174,0.041882,-0.094769,-0.107290,-0.141414,0.066270
creatinine_phosphokinase,-0.081584,-0.190741,1.000000,-0.009639,-0.044080,-0.070590,0.024463,-0.016408,0.059550,0.079791,0.002421,-0.009346,0.062728
diabetes,-0.101012,-0.012729,-0.009639,1.000000,-0.004850,-0.012732,0.092193,-0.046975,-0.089551,-0.157730,-0.147173,0.033726,-0.001943
ejection_fraction,0.060098,0.031557,-0.044080,-0.004850,1.000000,0.024445,0.072177,-0.011302,0.175902,-0.148386,-0.067315,0.041729,-0.268603
high_blood_pressure,0.093289,0.038182,-0.070590,-0.012732,0.024445,1.000000,0.049963,-0.004935,0.037109,-0.104615,-0.055711,-0.196439,0.079351
platelets,-0.052354,-0.043786,0.024463,0.092193,0.072177,0.049963,1.000000,-0.041198,0.062125,-0.125120,0.028234,0.010514,-0.049139
serum_creatinine,0.159187,0.052174,-0.016408,-0.046975,-0.011302,-0.004935,-0.041198,1.000000,-0.189095,0.006970,-0.027414,-0.149315,0.294278
serum_sodium,-0.045966,0.041882,0.059550,-0.089551,0.175902,0.037109,0.062125,-0.189095,1.000000,-0.027566,0.004813,0.087640,-0.195204
sex,0.065430,-0.094769,0.079791,-0.157730,-0.148386,-0.104615,-0.125120,0.006970,-0.027566,1.000000,0.445892,-0.015608,-0.004316


## **Zadatak 1: Modeliranje vremena provedenog u bolnici**

Potrebno je provesti regresijsku analizu za modeliranje broja dana provedenih u bolnici, među populacijom pacijenata. Počnimo s uzorkom koji je predstavljen našim skupom podataka. Zanimaju nas dva prediktora: dijabetes i visoki krvni tlak. Koristite ova dva prediktora kako biste modelirali broj dana u bolnici pomoću modela linearne regresije. 


Za početak, neka primjer modela bude:

vrijeme ~ C (dijabetes) + C (visoki krvni tlak).

Pomoću *ols()* funkcije iz biblioteke statsmodels definirajte model i procijenite parametre. 

In [17]:
from re import A
model = smf.ols(formula = 'time ~ C(diabetes) + C(high_blood_pressure)', data = df)
result = model.fit()
print(result.summary())
#R^2 - koeficijent odlucnosti - pokazuje nam da je 4% varijance zavisne varijablje objasnjen vezom izmedu zavisne i nezavisne varijable sto znaci da se regresijska linija nije
#priblizila stvarnim podacima



a = df.query('diabetes == 0 and high_blood_pressure == 0')['time'].mean()
b = df.query('diabetes == 1 and high_blood_pressure == 0')['time'].mean()
c = df.query('diabetes == 0 and high_blood_pressure == 1')['time'].mean()
d = df.query('diabetes == 1 and high_blood_pressure == 1')['time'].mean()
print(f'Prosjecno vrijeme provedeno u bolnici za pacijente koji nemaju dijabetes ni visoki krvni tlak: {a} dana.')
print(f'Prosjecno vrijeme provedeno u bolnici za pacijente koji imaju dijabetes ali nemaju visoki krvni tlak: {b} dana.')
print(f'Prosjecno vrijeme provedeno u bolnici za pacijente koji nemaju dijabetes ali imaju visoki krvni tlak: {c} dana.')
print(f'Prosjecno vrijeme provedeno u bolnici za pacijente koji imaju i dijabetes i visoki krvni tlak: {d} dana.')





                            OLS Regression Results                            
Dep. Variable:                   time   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     6.097
Date:                Tue, 11 Apr 2023   Prob (F-statistic):            0.00254
Time:                        18:54:36   Log-Likelihood:                -1718.9
No. Observations:                 299   AIC:                             3444.
Df Residuals:                     296   BIC:                             3455.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

Ispišite vrijednosti koeficijenata u modelu, p-vrijednosti F i t statističkih testova i $R_{adjusted}^2$. Komentirajte dobivene vrijednosti.

Interpretirajte dobiveni model prema vrijednostima koeficijenata. Navedite prosječno vrijeme provedeno u bolnici za:

1. Pacijente koji nemaju dijabetes ni visoki krvni tlak.
2. Pacijente koji imaju dijabetes, ali nemaju visoki krvni tlak.
3. Pacijente koji nemaju dijabetes, ali imaju visoki krvni tlak.
4. Pacijente koji imaju i dijabetes i visoki krvni tlak.

## **Zadatak 2: Modeliranje vremena provedenog u bolnici (dodavanje interakcijskih članova)**

U prethodnom modelu, moguće je zamijetiti kako osobe s povišenim krvnim tlakom kraće borave u bolnici, iako bi se moglo očekivati suprotno. Jedan od razloga zašto se ozbiljna stanja (poput hipertenzije) mogu povezati s kraćim vremenom provedenim u bolnici jest treći faktor - smrt. Pacijenti koji imaju obziljno stanje mogli bi provesti manje vremena u bolnici jer brzo umiru. Pokušajte steći bolji uvid u dani problem, ubacivanjem smrti kao još jednog prediktora u prethodnom modelu.

In [22]:
model_2 = smf.ols(formula = 'time ~ C(diabetes) + C(high_blood_pressure) + C(DEATH_EVENT)', data = df)
result_2 = model_2.fit()
print(result_2.summary())

a = df.query('diabetes == 0 and high_blood_pressure == 0')['time'].mean()
b = df.query('diabetes == 1 and high_blood_pressure == 0')['time'].mean()
c = df.query('diabetes == 0 and high_blood_pressure == 1')['time'].mean()
d = df.query('diabetes == 1 and high_blood_pressure == 1')['time'].mean()
print(f'Prosjecno vrijeme provedeno u bolnici za pacijente koji nemaju dijabetes ni visoki krvni tlak: {a} dana.')
print(f'Prosjecno vrijeme provedeno u bolnici za pacijente koji imaju dijabetes ali nemaju visoki krvni tlak: {b} dana.')
print(f'Prosjecno vrijeme provedeno u bolnici za pacijente koji nemaju dijabetes ali imaju visoki krvni tlak: {c} dana.')
print(f'Prosjecno vrijeme provedeno u bolnici za pacijente koji imaju i dijabetes i visoki krvni tlak: {d} dana.')



                            OLS Regression Results                            
Dep. Variable:                   time   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.296
Method:                 Least Squares   F-statistic:                     42.69
Date:                Tue, 11 Apr 2023   Prob (F-statistic):           6.12e-23
Time:                        19:09:42   Log-Likelihood:                -1671.0
No. Observations:                 299   AIC:                             3350.
Df Residuals:                     295   BIC:                             3365.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

Procijenite parametre pomoću funkcije *ols()* te interpretirajte dobivene koeficijente i $R_{adjusted}^2$. Opisuje li dobiveni model dani problem bolje od prethodnog modela? Je li smrt statistički značajan prediktor?

Razmislite jesu li smrt i visoki krvni tlak međusobno nezavisne varijable. Na temelju Vašeg zaključka, razmotrite potrebu za ubacivanjem njihovog interakcijskog člana u prethodni model. Procijenite parametre modela te interpretirajte dobivene koeficijente i $R_{adjusted}^2$. Opisuje li dobiveni model dani problem bolje od prethodnog modela?

In [24]:
df['high_blood_pressure_DEATH_EVENT'] = df['DEATH_EVENT'] * df['high_blood_pressure']
model_3 = smf.ols(formula = 'time ~ C(diabetes) + C(high_blood_pressure) + C(high_blood_pressure_DEATH_EVENT)', data = df)
result_3 = model_3.fit()
print(result_3.summary())

a = df.query('DEATH_EVENT == 0 and high_blood_pressure == 1')['time'].mean()
b = df.query('DEATH_EVENT == 1 and high_blood_pressure == 1')['time'].mean()

print(f'prosjecno vrijeme provedeno u bolnici kod pacijenata s visokim krvnim tlakom koji ne umiru tijekom boravka: {a}')
print(f'prosjecno vrijeme provedeno u bolnici kod pacijenata s visokim krvnim tlakom koji umiru tijekom boravka: {b}')



                            OLS Regression Results                            
Dep. Variable:                   time   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     15.38
Date:                Tue, 11 Apr 2023   Prob (F-statistic):           2.54e-09
Time:                        19:55:51   Log-Likelihood:                -1703.2
No. Observations:                 299   AIC:                             3414.
Df Residuals:                     295   BIC:                             3429.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

Navedite prosječno vrijeme provedeno u bolnici za:

1. Pacijente koji imaju visoki krvni tlak, a ne umiru tijekom razbolja praćenja.
2. Pacijente koji imaju visoki krvi tlak, a umiru tijekom razdoblja praćenja.

## **Zadatak 3. Logaritamske transformacije**

Prisjetite se da logaritamska transformacija izlazne varijable čini model multiplikativnim. Definirajte linearni model za predviđanje vremena provedenog u bolnici, a kao nezavisne varijable upotrijebite visoki krvni tlak, dijabetes i ishod smrti.

Zatim izvedite logaritamsku transformaciju izlazne varijable na način da kreirate dodatni stupac (*log_time*) u skupu podataka koji sadrži logaritme vrijednosti iz stupca *time*. Ponovno definirajte model regresije kao prethodni, no ovaj puta s novodefiniranim stupcem kao izlaznom varijablom. 

In [ ]:
# Mjesto za Vaš kod

Za varijablu visoki tlak izračunajte koliki je njezin aditivni učinak u prvom modelu te koliki je multiplikativni učinak u drugom modelu na temelju pripadnih koeficijenata. Drugim riječima, kako se mijenja vrijednost izlazne varijable u prvom, odnosno drugom modelu, ako pacijent ima visoki krvni tlak?

## **Zadatak 4: Logistička regresija: Modeliranje ishoda smrti**

Modelirajte (binarni) ishod smrti na temelju **svih** karakteristika pacijenata. Izuzmite jedino vrijeme provedeno u bolnici jer smo vidjeli kako je ono u velikoj korelaciji sa smrću.

Kao prvi korak, provedite standardizaciju prediktora pomoću z-vrijednosti. To će Vam olakšati tumačenje prediktora koji su vrlo različitih skala. 



In [ ]:
# Mjesto za Vaš kod

Umjesto linearne, koristite logističku regresiju za modeliranje binarnog ishoda smrti. Napomena: koristite *logit()* funkciju iz biblioteke statsmodels. 

In [ ]:
# Mjesto za Vaš kod

Protumačite dobivene koeficijente i $R_{adjusted}^2$. Koji su prediktori statistički značajni za modeliranje smrtnog ishoda? 
Pomoću *errorbar()* funkcije iz biblioteke matplotlib, vizualizirajte učinak svakog prediktora. Vrijednosti koeficijenata i standardne greške možete izvući iz modela regresije (*model.params.values* i *model.bse.values*). 

In [ ]:
# Mjesto za Vaš kod

### **Zadatak 4.1. Logistička regresija: Upravljanje prediktorima**

Primijetite da nezavisne varijable (one koje se koriste kao prediktori u regresijskom modelu) često koreliraju. Primjerice, serumski kreatinin važan je pokazatelj zdravlja bubrega. Pomoću modela logističke regresije procijenite učinak serumskog kreatinina na zdravlje pacijenta. Drugim riječima, definirajte model logističke regresije koji kao izlaznu varijablu ima ishod smrti, a kao prediktor razinu kreatinina u krvi. 

In [ ]:
# Mjesto za Vaš kod

Primijetite da je serumski kreatinin u značajnoj korelaciji s godinama. Što je osoba starija, to je razina kreatinina u krvi veća. Imajući ovo na umu, ponovite logističku regresiju sada koristeći serumski kreatinin i dob kao prediktorske varijable. Što očekujete da će se dogoditi s koeficijentom serumskog kreatinina? Hoće li se smanjiti ili povećati? Što se dogodilo s $R^2$ koeficijentom modela?

In [ ]:
# Mjesto za Vaš kod